In [5]:
import numpy as np
import pandas as pd
from numpy import genfromtxt
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [15]:
# Teste com a XOR
X = np.array([[0,0],[0,1],[1,0],[1,1]])
y = np.array([[0],[1],[1],[0]])
X_test = X
y = y+1
X_train = X

In [17]:
# n_classes = len(my_data.groupby('<class_name_space>''))
n_classes = max(y)[0]
sigma = 1

# x_test = [0.8,0.2]
# probability_map = {}
# X_test = np.array([[0.5,0.5],[0.8,0.2],[0.4,0.7]])
prob = np.zeros((X_test.shape[0],max(y)[0]))

## loop through all the X_test data (unclassified points)
for point in range(0,X_test.shape[0]):
    x_test = X_test[point,:] 
#     print(x_test)
    ## loop through i-th class
    for i in range(1,n_classes+1):
        X_i = X[np.where(y==i)[0],:]
        summ = 0.
        ## loop through j-th element of the i-th class
        for j in range(0,X_i.shape[0]):
            ## norm L2 (sum of squares)
            norm = 0.
            ## loop through k-th feature
            for k in range(0,X_i.shape[1]):
                ## TODO: covariance form X*M*X'
                norm = norm + (X_i[j,k]-x_test[k])*(X_i[j,k]-x_test[k])  
            ## Summation of Gaussians
            summ = summ + np.exp((-1/2)*(1/(2*sigma**2))*norm)
        ## Average of Parzen Window (normalization term)
        summ = summ/X_i.shape[0]
        ## Decision boundary
        prob[point, (i-1)] = summ

In [26]:
# # argmax of each 
y_prob = np.asarray([max(prob[i,:]) for i in range(0,prob.shape[0])])
y_bar = np.asarray([np.where(prob[ix,:] == max(prob[ix,:]))[0][0]+1 for ix in range(0,prob.shape[0])])

In [ ]:
def pnn(X,y, sigma):
    '''
    Probabilistic neural network 
    Using  Parzen-window
    X = train data
    y = test data
    '''
    ## z-score normalization/scaling
    for i in range(0,X.shape[1]):       
            y[i] = (y[i] - np.mean(X[:,i]))/np.std(X[:,i])
            X[:,i] =  (X[:,i] - np.mean(X[:,i]))/np.std(X[:,i])
    
    # ** PATTERN LAYER **
    Sum = .0
    for j in range(0,X.shape[0]):
        Sum = Sum + np.exp((-1/2)*(sum((X[j,:] - y)**2)/(sigma**2)))
    prob = Sum/X.shape[0]
    
    return prob

def pnn_classifier(X,y,sigma):
    '''
    
    X = n-dimension raw data
    y = class type prefered to be an int number
    '''
    
    r = np.random.permutation(X.shape[0])
    X = X[r,:]
    y = y[r,:]
    
    prob = np.zeros((X.shape[0],max(y)[0]))
    
    for i in range(0,X.shape[0]):
        x_i = X[i,:]
        x_new = np.delete(X, i, 0)
        y_new = np.delete(y, i, 0)
        
        for j in range(0,max(y)[0]):
            k = np.where(y == 1)[0]
            xtrain = x_new[k,:]
            prob[i, j] = pnn(xtrain, x_i,sigma) 
        
    dec =  np.zeros((X.shape[0],1))   
    
    for ix in range(0,X.shape[0]):
        argmax = np.where(prob[ix,:] == max(prob[ix,:]))[0][0]
        dec[ix] = prob[ix,argmax]-1
    
    return  dec, y, prob